In [1]:
import os
import random
import numpy as np
import torch
import torchvision
from torchsummary import summary

from fpga_nn_backend.datasets import *
from fpga_nn_backend.training import *
from fpga_nn_backend.evaluation import *
from fpga_nn_backend.models.relu_toy_models import *
from fpga_nn_backend.quantization import *
from fpga_nn_backend.fpga_simple.emulation import *
from fpga_nn_backend.fpga_simple.conversion import *
from fpga_nn_backend.utils import *

In [2]:
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version: 1.10.0
Torchvision Version: 0.11.1


In [3]:
data_dir = get_rel_pkg_path("dataset/")
weights_dir = get_rel_pkg_path("weights/")
session_dir = get_rel_pkg_path("sessions/")
models_dir = get_rel_pkg_path("models/")

In [4]:
dataset_type = ImageDatasetType.MNIST

In [5]:
orig_datasets = get_img_dataset(data_dir, dataset_type)

In [6]:
datasets = apply_img_transforms(orig_datasets, dataset_type, flatten=True)

In [7]:
dataloaders = get_dataloaders(datasets, 128, 128, num_workers=0)

In [8]:
input_dim = IMG_DATASET_TO_IMG_SIZE_FLAT[dataset_type]
num_classes = IMG_DATASET_TO_NUM_CLASSES[dataset_type]

model = ReLUToyModel(input_dim, num_classes, layer_dims=[])
model = model.to(device)

In [9]:
model = QuantWrapper(model)

In [10]:
summary(model, (input_dim,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1                  [-1, 784]               0
            Linear-2                   [-1, 10]           7,840
      ReLUToyModel-3                   [-1, 10]               0
       DeQuantStub-4                   [-1, 10]               0
Total params: 7,840
Trainable params: 7,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [11]:
# model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 11-18-2021 11-04-36 PM\Weights Best.pckl")))

In [12]:
# model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 11-20-2021 06-39-40 PM\Weights Best.pckl")))

In [13]:
model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 12-07-2021 11-47-53 PM\Weights Best.pckl")))

<All keys matched successfully>

In [14]:
criterion = get_loss()
criterion = criterion.to(device)

In [15]:
model.eval()
qconfig = torch.quantization.QConfig(
    activation=torch.quantization.MinMaxObserver.with_args(dtype=torch.quint8),
    weight=torch.quantization.MinMaxObserver.with_args(dtype=torch.qint8))
#qconfig = torch.quantization.get_default_qconfig('fbgemm')
model.qconfig = qconfig
#torch.quantization.fuse_modules(model.model.layers, [['0', '1'], ['2', '3'], ['4', '5'], ['6', '7']], inplace=True)
model = torch.quantization.prepare(model)
stats = get_dataloader_stats(dataloaders['test'], model, criterion, device)
model_int8 = torch.quantization.convert(model)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 22.48it/s]


In [16]:
print("Accuracy:", stats['acc'])

Accuracy: 0.9034


In [17]:
model_int8

QuantWrapper(
  (model): ReLUToyModel(
    (layers): Sequential(
      (0): QuantizedLinear(in_features=784, out_features=10, scale=0.10827332735061646, zero_point=125, qscheme=torch.per_tensor_affine)
    )
  )
  (quant): Quantize(scale=tensor([0.0039]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
)

In [18]:
model_int8.model.layers[0].weight().int_repr()

tensor([[ -8, -23, -24,  ..., -19,  -7,  -9],
        [-18, -23, -17,  ..., -20, -17, -19],
        [ -4, -24, -24,  ..., -14, -15,  -7],
        ...,
        [-19, -24,  -8,  ..., -17,  -7, -22],
        [ -4, -10,  -7,  ..., -16,  -7, -22],
        [-21, -12, -11,  ...,  -7,  -5, -14]], dtype=torch.int8)

In [19]:
model_int8.model.layers[0].weight()

tensor([[ 0.0207, -0.0310, -0.0344,  ..., -0.0172,  0.0241,  0.0172],
        [-0.0138, -0.0310, -0.0103,  ..., -0.0207, -0.0103, -0.0172],
        [ 0.0344, -0.0344, -0.0344,  ...,  0.0000, -0.0034,  0.0241],
        ...,
        [-0.0172, -0.0344,  0.0207,  ..., -0.0103,  0.0241, -0.0276],
        [ 0.0344,  0.0138,  0.0241,  ..., -0.0069,  0.0241, -0.0276],
        [-0.0241,  0.0069,  0.0103,  ...,  0.0241,  0.0310,  0.0000]],
       size=(10, 784), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.003444838570430875,
       zero_point=-14)

In [20]:
stats = get_dataloader_stats(dataloaders['test'], model_int8, criterion, device)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 25.76it/s]


In [21]:
print("Accuracy:", stats['acc'])

Accuracy: 0.9009


In [22]:
def safe_get_param(param):
    if param is not None:
        return param.int_repr().numpy()
    else:
        return None

In [23]:
converted_nn = ConvertedNN((1, 28, 28))

converted_nn.add_flatten_layer((1, 28, 28), 0, 0)

converted_nn.add_dense_layer((784,), (10,), 0, 1,
    weight=safe_get_param(model_int8.model.layers[0].weight()),
    bias=safe_get_param(model_int8.model.layers[0].bias()))

converted_nn.add_output_layer((10,), 1, 1)

In [24]:
model_int8.model.layers[0]

QuantizedLinear(in_features=784, out_features=10, scale=0.10827332735061646, zero_point=125, qscheme=torch.per_tensor_affine)

In [25]:
converted_nn.finalize()

In [26]:
converted_nn.get_layer_info()

{'layers': [{'layer_type': <ConverterLayerType.FLATTEN: 5>,
   'input_shapes': ((1, 28, 28),),
   'output_shape': (784,),
   'output_size': 784,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': None,
   'metadata': None},
  {'layer_type': <ConverterLayerType.DENSE: 0>,
   'input_shapes': ((784,),),
   'output_shape': (10,),
   'output_size': 10,
   'stack_input_indices': (0,),
   'stack_output_index': 1,
   'parameters': {'weight': 0},
   'metadata': {'has_bias': False}},
  {'layer_type': <ConverterLayerType.OUTPUT: 7>,
   'input_shapes': ((10,),),
   'output_shape': (10,),
   'output_size': 10,
   'stack_input_indices': (1,),
   'stack_output_index': 1,
   'parameters': None,
   'metadata': None}]}

In [27]:
with open("test.coe", 'w') as f:
    f.write(converted_nn.generate_parameter_coe())

In [28]:
emulator = FPGAEmulator(converted_nn, bram_reserved_size=303000)

In [29]:
emulator.exec_info

{'output_shape': (10,),
 'output_size': 10,
 'input_shape': (1, 28, 28),
 'input_size': 784,
 'inital_input_addr': 0,
 'layers': [{'layer_type': <LayerType.INPUT_RESET: 7>,
   'config': {'output_base_addr': 784, 'n_size': 11}},
  {'layer_type': <LayerType.DENSE: 0>,
   'config': {'has_bias': None,
    'input_base_addr': 0,
    'weight_base_addr': 0,
    'bias_base_addr': 0,
    'output_base_addr': 784,
    'm_size': 10,
    'chw_size': 784}},
  {'layer_type': <LayerType.OUTPUT: 6>,
   'config': {'output_base_addr': 784, 'n_size': 10}}]}

In [30]:
index = 1
for imgs, labels in dataloaders['test']:
    break
data = (imgs[index] * 255).numpy()
data = data.astype(np.uint8)
print(data)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 116 125 171 255 255 150  93   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0 169 253 253 253 253
 253 253 218  30   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 169 253 253 253 213 142 176 253 253 122   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  52 250 253 210  32
  12   0   6 206 253 140   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  77 251 210  25   0   0   0 122 248 253  65   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  31  18
   0   0   0   0 209 253 253  65   0   0   0   0   

In [31]:
parameters = converted_nn.parameters_info['parameters']
print([p.shape for p in parameters])

def dense_no_bias(w, i):
    pass

def relu(i):
    return np.maximum(0, i)

[(10, 784)]


In [32]:
inputs_per_layer = {}
outputs_per_layer = {}
def get_output(name):
    def hook(model, input, output):
        inputs_per_layer[name] = input
        outputs_per_layer[name] = output.detach()
    return hook

hooks = {}
for name, module in model_int8.named_modules():
    hooks[name] = module.register_forward_hook(get_output(name))

In [33]:
batch = imgs[None, index, :]
model_out = model_int8(batch)

In [56]:
parameters[0][0] = -8

In [45]:
outputs_per_layer.keys()

dict_keys(['quant', 'model.layers.0', 'model.layers', 'model', 'dequant', ''])

In [46]:
inputs_per_layer['model.layers.0'][0].int_repr()

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 116, 125, 171, 255,
         255, 150,  93,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 253,
         253, 253, 218,  30,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 213, 142,
         176, 253, 253, 122,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,  52, 250

In [47]:
outputs_per_layer['model.layers.0'].int_repr()

tensor([[147, 112, 185, 151,  70, 151, 161,  66, 144,  79]], dtype=torch.uint8)

In [48]:
model_int8.model.layers[0].weight()

tensor([[ 0.0207, -0.0310, -0.0344,  ..., -0.0172,  0.0241,  0.0172],
        [-0.0138, -0.0310, -0.0103,  ..., -0.0207, -0.0103, -0.0172],
        [ 0.0344, -0.0344, -0.0344,  ...,  0.0000, -0.0034,  0.0241],
        ...,
        [-0.0172, -0.0344,  0.0207,  ..., -0.0103,  0.0241, -0.0276],
        [ 0.0344,  0.0138,  0.0241,  ..., -0.0069,  0.0241, -0.0276],
        [-0.0241,  0.0069,  0.0103,  ...,  0.0241,  0.0310,  0.0000]],
       size=(10, 784), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.003444838570430875,
       zero_point=-14)

In [38]:
emulator.execute(data)

{'layer_type': <LayerType.INPUT_RESET: 7>, 'config': {'output_base_addr': 784, 'n_size': 11}}
LayerType.INPUT_RESET LayerStep.INIT_BIAS
linear_mac_loop_input_addr: 0
linear_mac_loop_weight_addr: 0
linear_mac_loop_output_addr: 0
linear_mac_loop_started: 0
linear_mac_loop_ready_out: 0
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [0]
---
bram0_read_addr: 0
bram0_read_enable: 0
bram0_read_out: 0
bram1_read_enable: 0
bram1_read_addr: 0
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 0
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 0
weights_mac: [0]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.INPUT_RESET LayerStep.INIT_BIAS
linear_mac_loop_input_addr: 0
linear_mac_loop_weight_addr: 0
linear_mac_loop_output_addr: 0
linear_mac_loop_started: 0
linear_ma

bram0_read_addr: 0
bram0_read_enable: 0
bram0_read_out: -8
bram1_read_enable: 0
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 0
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 0
weights_mac: [-8]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 1
linear_mac_loop_weight_addr: 1
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 1
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 0
bram0_read_enable: 0
bram0_read_out: -8
bram1_read_enable: 0
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 0
bram1_write_val: 0
---
mac_ready_in: 1
mac_done_out: 0
weights_mac: [-8]
inputs_mac: [0]
biases_mac:

linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 1
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 1
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 2
bram0_read_enable: 0
bram0_read_out: -24
bram1_read_enable: 0
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-24]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 3
linear_mac_loop_weight_addr: 3
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 1
linear_mac_l

linear_mac_loop_input_addr: 5
linear_mac_loop_weight_addr: 5
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 2
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 5
bram0_read_enable: 0
bram0_read_out: -13
bram1_read_enable: 1
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-13]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 5
linear_mac_loop_weight_addr: 5
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 1
linear_mac_l

bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-19]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 7
linear_mac_loop_weight_addr: 7
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 1
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 1
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 7
bram0_read_enable: 0
bram0_read_out: -19
bram1_read_enable: 0
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-19]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 8
l

...

linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 82
bram0_read_enable: 1
bram0_read_out: -22
bram1_read_enable: 1
bram1_read_addr: 82
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-22]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 82
linear_mac_loop_weight_addr: 82
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 2
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 82
bram0_read_enable: 0
bram0_read_out: -17
bram1_read_enable: 1
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_va

linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 1
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 85
bram0_read_enable: 1
bram0_read_out: -8
bram1_read_enable: 1
bram1_read_addr: 85
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-8]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 85
linear_mac_loop_weight_addr: 85
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 2
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 0
linear_mac_l

mac_done_out: 0
weights_mac: [-7]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 88
linear_mac_loop_weight_addr: 88
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 1
linear_mac_loop_write_lane_index: 1
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 87
bram0_read_enable: 0
bram0_read_out: -7
bram1_read_enable: 0
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 1
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-7]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 88
linear_mac_loop_weight_addr: 88
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_ma

linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 1
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 1
linear_mac_loop_write_step: 0
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [784]
---
bram0_read_addr: 91
bram0_read_enable: 0
bram0_read_out: -5
bram1_read_enable: 0
bram1_read_addr: 784
bram1_read_out: 0
bram1_write_enable: 0
bram1_write_addr: 784
bram1_write_val: 0
---
mac_ready_in: 0
mac_done_out: 1
weights_mac: [-5]
inputs_mac: [0]
biases_mac: [0]
outputs_mac: [0]
---
LayerType.DENSE LayerStep.MAC_OUTPUT
linear_mac_loop_input_addr: 92
linear_mac_loop_weight_addr: 92
linear_mac_loop_output_addr: 784
linear_mac_loop_started: 1
linear_mac_loop_ready_out: 1
linear_mac_loop_done_out: 0
linear_mac_loop_start_ready_in: 0
linear_mac_loop_next_ready_in: 0
linear_mac_loop_read_step: 0
linear_mac_loop_read_lane_index: 0
linear_mac_loop_write_step: 1
linear_mac_loop_write_lane_index: 0
linear_mac_loop_output_addrs: [

ValueError: 

In [50]:
w = parameters[0]

In [51]:
data = (inputs_per_layer['model.layers.0'][0].int_repr()).numpy()[0, :]

In [57]:
out = (w.astype(np.int32) @ data.astype(np.int32))

In [58]:
print(out)

[-230800 -517266   62549 -199621 -899189 -196677 -112234 -939653 -249692
 -779382]


In [59]:
np.argsort(outputs_per_layer['model.layers.0'].int_repr().numpy())

array([[7, 4, 9, 1, 8, 0, 3, 5, 6, 2]], dtype=int64)

In [60]:
np.argsort(out)

array([7, 4, 9, 1, 8, 0, 3, 5, 6, 2], dtype=int64)

In [ ]:
labels[index]

In [65]:
sample_img = data
packed_array = "{"
for i, s in enumerate(sample_img):
    packed_array = packed_array + f"32'sh{hex(s)[2:]}"
    if i != len(sample_img) - 1:
        packed_array = packed_array + ", "
packed_array = packed_array + "};"
print(packed_array)

{32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh74, 32'sh7d, 32'shab, 32'shff, 32'shff, 32'sh96, 32'sh5d, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sh0, 32'sha9, 32'shfd, 32'shf

In [ ]:
np.all(data >= 0)

In [40]:
data[94]

116

In [64]:
data

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 116, 125, 171, 255, 255, 150,  93,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0, 169, 253, 253, 253, 253, 253, 253, 218,  30,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0, 169, 253, 253, 253, 213, 142, 176, 253,
       253, 122,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,  52, 250, 253, 210,  32,  1